<a href="https://colab.research.google.com/github/Anirudhvr02/da6401_assignment1/blob/main/DA6401_Assignment1_Sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

class NeuralLayer:
    """
    Represents an individual layer in a neural network.
    """
    def __init__(self, index, n_input, n_neurons, function=None, weights=None, bias=None, method="random"):
        """
        Initializes a neural network layer with specified parameters.

        Args:
            index (int): Position of the layer in the network.
            n_input (int): Number of incoming connections (input features).
            n_neurons (int): Number of neurons in the layer.
            function (str, optional): Activation function to be used (default: 'sigmoid').
            weights (numpy.ndarray, optional): Predefined weight matrix.
                                               If not provided, it will be initialized based on the given method.
            bias (numpy.ndarray, optional): Bias for the neurons.
                                            If not provided, it will be initialized randomly.
            method (str, optional): Strategy for initializing weights (default: 'random').

        """
        self.index = index
        self.function = function or 'sigmoid'  # Default to sigmoid if no function provided
        self.weights = weights if weights is not None else self.initialize_weights(method, n_input, n_neurons)
        self.bias = bias if bias is not None else np.random.randn(n_neurons)  # Initialize bias using normal distribution
        self.activation = None

        # Initialize placeholders for backpropagation
        self.error = None
        self.delta = None

        # Gradients for weights and biases (used for optimization)
        self.d_weights = np.zeros((n_input, n_neurons))
        self.d_bias = np.zeros(n_neurons)

        # Momentum and accumulated gradients for advanced optimization methods
        self.h_weights = np.zeros((n_input, n_neurons))
        self.h_bias = np.zeros(n_neurons)
        self.m_weights = np.zeros((n_input, n_neurons))
        self.m_bias = np.zeros(n_neurons)

    def initialize_weights(self, method, n_input, n_neurons):
        """
        Initializes the weight matrix based on the chosen method.

        Args:
            method (str): Weight initialization method ('Xavier' or 'random').
            n_input (int): Number of input features.
            n_neurons (int): Number of neurons in the layer.

        Returns:
            numpy.ndarray: Initialized weight matrix.
        """
        if method == "Xavier":
            # Xavier initialization scales the weights to avoid vanishing/exploding gradients
            limit = np.sqrt(2 / (n_input + n_neurons))
            return np.random.uniform(-limit, limit, (n_input, n_neurons))
        # Default to normal distribution if no method is specified
        return np.random.randn(n_input, n_neurons)

    def activate(self, x):
        """
        Computes the output of the layer after applying the activation function.

        Args:
            x (numpy.ndarray): Input values (can be from previous layer or direct input).

        Returns:
            numpy.ndarray: Activated output values.
        """
        z = np.dot(x, self.weights) + self.bias  # Weighted sum of inputs and bias
        self.activation = self._apply_activation(z)
        return self.activation

    def _apply_activation(self, r):
        """
        Applies the activation function to the computed output.

        Args:
            r (numpy.ndarray): Raw output values before activation.

        Returns:
            numpy.ndarray: Output values after applying the activation function.
        """
        if self.function == 'sigmoid':
            # Sigmoid squashes output between 0 and 1
            return 1 / (1 + np.exp(-r))
        elif self.function == 'tanh':
            # Tanh scales output between -1 and 1
            return np.tanh(r)
        elif self.function == 'ReLu':
            # ReLU returns zero for negative values
            return np.maximum(0, r)
        elif self.function == 'softmax':
            # Softmax normalizes output to a probability distribution
            max_r = np.max(r, axis=1, keepdims=True)
            exp_r = np.exp(r - max_r)  # Subtracting max for numerical stability
            return exp_r / np.sum(exp_r, axis=1, keepdims=True)
        return r  # Linear activation (no transformation)

    def apply_activation_derivative(self, z):
        """
        Calculates the derivative of the activation function.

        Args:
            z (numpy.ndarray): Activated output values.

        Returns:
            numpy.ndarray: Derivative of the activation function.
        """
        if self.function == 'sigmoid':
            # Derivative of sigmoid: σ(z) * (1 - σ(z))
            return z * (1 - z)
        elif self.function == 'tanh':
            # Derivative of tanh: 1 - tanh²(z)
            return 1 - z**2
        elif self.function == 'ReLu':
            # Derivative of ReLU is 1 for positive inputs, 0 for negative
            return np.where(z > 0, 1, 0)
        elif self.function == 'softmax':
            # Derivative of softmax (for single output): diagonal form
            return np.diag(z) - np.outer(z, z)
        # Default to identity (no transformation)
        return np.ones_like(z)

    def __str__(self):
        """
        Returns a formatted string showing the layer configuration.

        Returns:
            str: String describing the layer.
        """
        return f'Neural Layer: Index={self.index}, Shape={self.weights.shape}, Activation={self.function}'


In [3]:
import numpy as np

class NeuralNetwork:
    def __init__(self, config):
        """
        Initialize a neural network based on the provided configuration.

        Args:
        - config (dict): Configuration dictionary containing network parameters.
          Keys:
            - input_size (int): Number of input features.
            - num_layers (int): Number of hidden layers.
            - output_size (int): Number of output neurons.
            - hidden_size (int): Number of neurons in hidden layers.
            - activation (str): Activation function for hidden layers.
            - output_activation (str): Activation function for output layer.
            - criterion (str): Loss function type ('cross_entropy', 'mse', etc.)
            - weight_initialization (str): Method for weight initialization ('random', 'Xavier').
        """
        # Helper function to get a value from config or return default
        def get_value(key, default):
            return config.get(key, default)

        # Store configuration values
        self.criterion = get_value('criterion', 'cross_entropy')
        self.weight_initialization = get_value('weight_initialization', 'random')

        # Initialize the objective function for loss calculation
        self.c = ObjectiveFunction(method=self.criterion)

        # Initialize layers based on config
        self.layers = []
        self.add_layers(
            input_size=config['input_size'],
            hidden_layers=config['num_layers'],
            output_size=config['output_size'],
            neurons=config['hidden_size'],
            activation=config['activation'],
            output_activation=config['output_activation']
        )

    def forward(self, x):
        """
        Perform forward pass through the neural network.

        Args:
        - x (numpy.ndarray): Input data of shape (batch_size, input_size).

        Returns:
        - numpy.ndarray: Output of the network (batch_size, output_size).
        """
        for layer in self.layers:
            x = layer.activate(x)
        return x

    def backward(self, x, y, y_hat):
        """
        Perform backward pass (backpropagation) through the neural network.

        Args:
        - x (numpy.ndarray): Input data (batch_size, input_size).
        - y (numpy.ndarray): True labels (batch_size, output_size).
        - y_hat (numpy.ndarray): Predicted labels (batch_size, output_size).
        """
        batch_size = y.shape[0]

        # Backpropagate from the output layer to the first hidden layer
        for i in reversed(range(len(self.layers))):
            layer = self.layers[i]

            if i == len(self.layers) - 1:  # Output layer
                layer.error = self.c.get_derivative(y, y_hat)

                # Handle softmax derivatives separately due to multi-class output
                output_derivative_matrix = []
                for j in range(batch_size):
                    derivative = layer.apply_activation_derivative(y_hat[j])
                    output_derivative_matrix.append(np.matmul(layer.error[j], derivative))

                layer.delta = np.array(output_derivative_matrix)
            else:
                # Hidden layers: Backpropagate the error from the next layer
                next_layer = self.layers[i + 1]
                layer.error = np.matmul(next_layer.delta, next_layer.weights.T)
                layer.delta = layer.error * layer.apply_activation_derivative(layer.activation)

        # Update gradients for each layer
        for i, layer in enumerate(self.layers):
            activation_input = np.atleast_2d(x if i == 0 else self.layers[i - 1].activation)
            layer.d_weights = np.matmul(activation_input.T, layer.delta) / batch_size
            layer.d_bias = np.sum(layer.delta, axis=0) / batch_size

    def add_layers(self, input_size, hidden_layers, output_size, neurons, activation, output_activation):
        """
        Add layers to the neural network.

        Args:
        - input_size (int): Number of input features.
        - hidden_layers (int): Number of hidden layers.
        - output_size (int): Number of output neurons.
        - neurons (int): Number of neurons in each hidden layer.
        - activation (str): Activation function for hidden layers.
        - output_activation (str): Activation function for the output layer.
        """
        # Add hidden layers and output layer
        for i in range(hidden_layers + 1):
            if i == 0:
                # First hidden layer connects to input layer
                n_input = input_size
            else:
                # Subsequent hidden layers connect to previous layer
                n_input = neurons

            # Last layer is output layer, others are hidden layers
            n_neurons = output_size if i == hidden_layers else neurons
            act_func = output_activation if i == hidden_layers else activation

            layer = NeuralLayer(
                index=i + 1,
                n_input=n_input,
                n_neurons=n_neurons,
                function=act_func,
                method=self.weight_initialization
            )
            self.layers.append(layer)

    def __str__(self):
        """
        Return a string representation of the neural network.

        Returns:
        - str: String summarizing the network structure.
        """
        layer_summary = '\n'.join(str(layer) for layer in self.layers)
        return f"Neural Network Summary:\n{layer_summary}"


In [4]:
import numpy as np

class ObjectiveFunction:
    def __init__(self, method):
        """
        Initialize the objective function with a specific loss calculation method.

        Args:
            method (str): Specifies the type of loss function to use.
                         Supported values:
                         - 'cross_entropy' -> Cross-entropy loss
                         - 'mean_squared_error' -> Mean squared error loss
        """
        self.method = method

    def get_loss(self, y, y_hat):
        """
        Calculate the loss value between the true and predicted values.

        This method dynamically selects the appropriate loss function
        based on the 'method' provided during initialization.

        Args:
            y (numpy.ndarray): Array of true values.
            y_hat (numpy.ndarray): Array of predicted values.

        Returns:
            float: Computed loss value.
        """
        if self.method == "cross_entropy":
            return self._cross_entropy_loss(y, y_hat)
        elif self.method == "mean_squared_error":
            return self._mean_squared_error(y, y_hat)
        else:
            raise ValueError(f"Unsupported loss function: {self.method}")

    def get_derivative(self, y, y_hat):
        """
        Compute the derivative (gradient) of the loss function.

        This method returns the gradient based on the selected loss type.
        The gradient is useful for backpropagation in neural networks.

        Args:
            y (numpy.ndarray): True values.
            y_hat (numpy.ndarray): Predicted values.

        Returns:
            numpy.ndarray: Gradient of the loss function.
        """
        if self.method == "cross_entropy":
            return self._cross_entropy_derivative(y, y_hat)
        elif self.method == "mean_squared_error":
            return self._mean_squared_error_derivative(y, y_hat)
        else:
            raise ValueError(f"Unsupported loss function: {self.method}")

    def _mean_squared_error(self, y, y_hat):
        """
        Compute the Mean Squared Error (MSE) between true and predicted values.

        MSE measures the average of the squared differences
        between predicted and actual values.

        Args:
            y (numpy.ndarray): True values.
            y_hat (numpy.ndarray): Predicted values.

        Returns:
            float: MSE value.
        """
        diff = y - y_hat
        mse = np.mean(np.square(diff))  # More efficient and cleaner
        return mse

    def _mean_squared_error_derivative(self, y, y_hat):
        """
        Compute the gradient of the Mean Squared Error (MSE) loss function.

        This derivative shows how much each predicted value
        deviates from the actual value.

        Args:
            y (numpy.ndarray): True values.
            y_hat (numpy.ndarray): Predicted values.

        Returns:
            numpy.ndarray: Gradient of the MSE loss function.
        """
        return y_hat - y

    def _cross_entropy_loss(self, y, y_hat):
        """
        Compute the Cross-Entropy Loss between true and predicted values.

        Cross-entropy loss is commonly used in classification problems.
        It measures how similar the predicted probability distribution
        is to the actual distribution.

        Args:
            y (numpy.ndarray): True values (one-hot encoded).
            y_hat (numpy.ndarray): Predicted probabilities.

        Returns:
            float: Cross-entropy loss value.
        """
        epsilon = 1e-12  # Small value to avoid log(0) errors
        y_hat = np.clip(y_hat, epsilon, 1. - epsilon)  # Ensure valid probability values
        loss = -np.mean(np.sum(y * np.log(y_hat), axis=1))  # Average over samples
        return loss

    def _cross_entropy_derivative(self, y, y_hat):
        """
        Compute the gradient of the Cross-Entropy Loss.

        The derivative helps in adjusting the model’s weights
        to reduce classification error.

        Args:
            y (numpy.ndarray): True values (one-hot encoded).
            y_hat (numpy.ndarray): Predicted probabilities.

        Returns:
            numpy.ndarray: Gradient of the cross-entropy loss.
        """
        epsilon = 1e-12
        y_hat = np.clip(y_hat, epsilon, 1. - epsilon)  # Prevent division by zero
        gradient = -y / y_hat
        return gradient


In [5]:
import numpy as np

class Optimizer:
    def __init__(self, nn, config=None):
        """
        Initialize the optimizer.

        Args:
            nn (NeuralNetwork): Neural network instance to optimize.
            config (dict): Configuration dictionary containing optimizer parameters.
        """
        # Assign neural network and optimizer settings
        self.nn = nn
        self.lr = config.get('learning_rate', 0.001)
        self.optimizer = config.get('optimizer', 'sgd')

        # Assign optimization hyperparameters with sensible defaults
        self.beta = config.get('beta', 0.9)
        self.epsilon = config.get('epsilon', 1e-8)
        self.beta1 = config.get('beta1', 0.9)
        self.beta2 = config.get('beta2', 0.999)
        self.decay = config.get('weight_decay', 0.0)

        # Initialize timestep for Adam/Nadam updates
        self.timestep = 0

    def step(self):
        """
        Perform an optimization step based on the selected optimizer type.
        """
        self.timestep += 1

        if self.optimizer == "sgd":
            self.sgd()
        elif self.optimizer == "momentum":
            self.momentum()
        elif self.optimizer == "nag":
            self.nag()
        elif self.optimizer == "rmsprop":
            self.rmsprop()
        elif self.optimizer == "adam":
            self.adam()
        elif self.optimizer == "nadam":
            self.nadam()

    def sgd(self):
        """
        Perform stochastic gradient descent (SGD) optimization.
        Updates weights and biases based on gradients.
        """
        for layer in self.nn.layers:
            # Update weights with L2 regularization (weight decay)
            layer.weights -= self.lr * (layer.d_weights + self.decay * layer.weights)
            layer.bias -= self.lr * (layer.d_bias + self.decay * layer.bias)

    def momentum(self):
        """
        Perform momentum-based gradient descent.
        Momentum helps accelerate gradient descent by using a moving average of gradients.
        """
        for layer in self.nn.layers:
            # Update momentum terms using the beta coefficient
            layer.h_weights = self.beta * layer.h_weights + (1 - self.beta) * layer.d_weights
            layer.h_bias = self.beta * layer.h_bias + (1 - self.beta) * layer.d_bias

            # Apply updates to weights and biases with regularization
            layer.weights -= self.lr * (layer.h_weights + self.decay * layer.weights)
            layer.bias -= self.lr * (layer.h_bias + self.decay * layer.bias)

    def nag(self):
        """
        Perform Nesterov accelerated gradient (NAG) optimization.
        Adjusts gradients based on a look-ahead step.
        """
        for layer in self.nn.layers:
            # Compute look-ahead gradient
            look_ahead_weights = layer.weights - self.beta * layer.h_weights
            look_ahead_bias = layer.bias - self.beta * layer.h_bias

            # Update momentum terms based on look-ahead values
            layer.h_weights = self.beta * layer.h_weights + (1 - self.beta) * layer.d_weights
            layer.h_bias = self.beta * layer.h_bias + (1 - self.beta) * layer.d_bias

            # Apply updates using look-ahead gradients
            layer.weights = look_ahead_weights - self.lr * (layer.d_weights + self.decay * look_ahead_weights)
            layer.bias = look_ahead_bias - self.lr * (layer.d_bias + self.decay * look_ahead_bias)

    def rmsprop(self):
        """
        Perform RMSprop optimization.
        Adjusts learning rate based on a moving average of squared gradients.
        """
        for layer in self.nn.layers:
            # Update running average of squared gradients
            layer.h_weights = self.beta * layer.h_weights + (1 - self.beta) * (layer.d_weights ** 2)
            layer.h_bias = self.beta * layer.h_bias + (1 - self.beta) * (layer.d_bias ** 2)

            # Apply RMSprop update with regularization
            layer.weights -= self.lr * (layer.d_weights / (np.sqrt(layer.h_weights) + self.epsilon) + self.decay * layer.weights)
            layer.bias -= self.lr * (layer.d_bias / (np.sqrt(layer.h_bias) + self.epsilon) + self.decay * layer.bias)

    def adam(self):
        """
        Perform Adam optimization.
        Combines momentum and RMSprop.
        """
        for layer in self.nn.layers:
            # Update biased first moment estimate (momentum)
            layer.m_weights = self.beta1 * layer.m_weights + (1 - self.beta1) * layer.d_weights
            layer.m_bias = self.beta1 * layer.m_bias + (1 - self.beta1) * layer.d_bias

            # Update biased second moment estimate (squared gradients)
            layer.h_weights = self.beta2 * layer.h_weights + (1 - self.beta2) * (layer.d_weights ** 2)
            layer.h_bias = self.beta2 * layer.h_bias + (1 - self.beta2) * (layer.d_bias ** 2)

            # Bias correction terms
            m_hat_weights = layer.m_weights / (1 - self.beta1 ** self.timestep)
            m_hat_bias = layer.m_bias / (1 - self.beta1 ** self.timestep)
            v_hat_weights = layer.h_weights / (1 - self.beta2 ** self.timestep)
            v_hat_bias = layer.h_bias / (1 - self.beta2 ** self.timestep)

            # Apply Adam updates
            layer.weights -= self.lr * (m_hat_weights / (np.sqrt(v_hat_weights) + self.epsilon) + self.decay * layer.weights)
            layer.bias -= self.lr * (m_hat_bias / (np.sqrt(v_hat_bias) + self.epsilon) + self.decay * layer.bias)

    def nadam(self):
        """
        Perform NAdam (Nesterov Adam) optimization.
        Combines Nesterov momentum with Adam-style moment estimation.
        """
        for layer in self.nn.layers:
            # Update first and second moment estimates
            layer.m_weights = self.beta1 * layer.m_weights + (1 - self.beta1) * layer.d_weights
            layer.m_bias = self.beta1 * layer.m_bias + (1 - self.beta1) * layer.d_bias
            layer.h_weights = self.beta2 * layer.h_weights + (1 - self.beta2) * (layer.d_weights ** 2)
            layer.h_bias = self.beta2 * layer.h_bias + (1 - self.beta2) * (layer.d_bias ** 2)

            # Bias correction terms
            m_hat_weights = layer.m_weights / (1 - self.beta1 ** self.timestep)
            m_hat_bias = layer.m_bias / (1 - self.beta1 ** self.timestep)
            v_hat_weights = layer.h_weights / (1 - self.beta2 ** self.timestep)
            v_hat_bias = layer.h_bias / (1 - self.beta2 ** self.timestep)

            # Nesterov-style correction
            nadam_weights = self.beta1 * m_hat_weights + (1 - self.beta1) * layer.d_weights / (1 - self.beta1 ** self.timestep)
            nadam_bias = self.beta1 * m_hat_bias + (1 - self.beta1) * layer.d_bias / (1 - self.beta1 ** self.timestep)

            # Apply NAdam updates
            layer.weights -= self.lr * (nadam_weights / (np.sqrt(v_hat_weights) + self.epsilon) + self.decay * layer.weights)
            layer.bias -= self.lr * (nadam_bias / (np.sqrt(v_hat_bias) + self.epsilon) + self.decay * layer.bias)


In [6]:
!pip install wandb -qU

In [ ]:
from keras.datasets import fashion_mnist
import wandb
from sklearn.model_selection import train_test_split
import numpy as np

# Sweep configuration for hyperparameter tuning using W&B
sweep_config = {
    'method': 'random',
    'name': 'Q4_SWEEP',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'num_layers': {'values': [1, 2, 3]},
        'hidden_size': {'values': [32, 64, 128]},
        'input_size': {'value': 784},
        'output_size': {'value': 10},
        'hidden_layers': {'values': [3, 4, 5]},
        'neurons': {'values': [32, 64, 128]},
        'activation': {'values': ['sigmoid', 'tanh', 'relu']},
        'output_activation': {'value': 'softmax'},
        'learning_rate': {'values': [1e-3, 1e-4]},
        'weight_decay': {'values': [0, 0.0005, 0.000005]},
        'epochs': {'values': [5, 10]},  # Changed to 'values' so a value is chosen from the list
        'optimizer': {'values': ['sgd', 'momentum', 'nag', 'rmsprop', 'adam', 'nadam']},
        'batch_size': {'values': [16, 32, 64]},
        'weight_initialization': {'values': ['xavier', 'random']},
        'beta': {'values': [0.7, 0.8, 0.9]},
        'beta1': {'value': 0.9},
        'beta2': {'value': 0.9999},
        'epsilon': {'value': 1e-8},
        'criterion': {'value': 'cross_entropy'}
    }
}

def accuracy(y, y_hat):
    correct_predictions = np.sum(np.argmax(y_hat, axis=1) == np.argmax(y, axis=1))
    total_samples = y.shape[0]
    return correct_predictions / total_samples

def train():
    # Load Fashion MNIST dataset
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    # Preprocess data
    train_images = train_images.reshape(train_images.shape[0], 784) / 255.0
    X_test = test_images.reshape(test_images.shape[0], 784) / 255.0

    # One-hot encode labels
    train_labels = np.eye(10)[train_labels]
    Y_test = np.eye(10)[test_labels]

    # Split into training and validation sets (90% train, 10% validation)
    X_train, X_val, Y_train, Y_val = train_test_split(train_images, train_labels, test_size=0.1, shuffle=True, random_state=27)

    # Authenticate and initialize W&B project
    wandb.login(key='b7ffff6aaec53209d700507bd3c9ebaf8e9d2c48')
    wandb.init(project="DA6401_Assignment1")

    # Debug: Print the current config to see which parameters were received
    config = wandb.config
    print("Wandb config:", dict(config))

    # Safely retrieve parameters using getattr with defaults
    hidden_layers = getattr(config, 'hidden_layers', 3)
    neurons = getattr(config, 'neurons', 64)
    activation = getattr(config, 'activation', 'relu')
    learning_rate = getattr(config, 'learning_rate', 1e-3)
    batch_size = getattr(config, 'batch_size', 32)
    optimizer_str = getattr(config, 'optimizer', 'adam')
    decay = getattr(config, 'decay', 0)
    weight_initialization = getattr(config, 'weight_initialization', 'xavier')
    epochs = getattr(config, 'epochs', 5)  # default to 5 if missing

    # Set a custom run name based on configuration parameters
    wandb.run.name = (f"hl_{hidden_layers}_nu_{neurons}_"
                      f"ac_{activation}_lr_{learning_rate}_"
                      f"bs_{batch_size}_opt_{optimizer_str}_"
                      f"de_{decay}_init_{weight_initialization}")

    # Initialize neural network and optimizer (make sure these classes are defined)
    nn = NeuralNetwork(config)
    optimizer = Optimizer(nn=nn, config=config)
    criterion = ObjectiveFunction(method=config.criterion)

    # Training loop
    for epoch in range(epochs):
        for batch_start in range(0, X_train.shape[0], batch_size):
            X_batch = X_train[batch_start:batch_start + batch_size]
            Y_batch = Y_train[batch_start:batch_start + batch_size]

            Y_hat_batch = nn.forward(X_batch)
            nn.backward(X_batch, Y_batch, Y_hat_batch)
            optimizer.step()

        optimizer.timestep += 1

        # Evaluate on training data
        Y_hat_train = nn.forward(X_train)
        train_loss = criterion.get_loss(Y_train, Y_hat_train)
        train_accuracy = accuracy(Y_train, Y_hat_train)

        # Evaluate on validation data
        Y_hat_val = nn.forward(X_val)
        val_loss = criterion.get_loss(Y_val, Y_hat_val)
        val_accuracy = accuracy(Y_val, Y_hat_val)

        # Log metrics to W&B
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_accuracy,
            "val_loss": val_loss,
            "val_accuracy": val_accuracy
        })

    # Evaluate on test data
    Y_hat_test = nn.forward(X_test)
    test_loss = criterion.get_loss(Y_test, Y_hat_test)
    test_accuracy = accuracy(Y_test, Y_hat_test)

    wandb.log({
        "test_loss": test_loss,
        "test_accuracy": test_accuracy
    })

    # Finish the W&B run
    wandb.finish()

# Create and run the sweep agent
wandb_id = wandb.sweep(sweep_config, project="DA6401_Assignment1")
wandb.agent(wandb_id, function=train, count=200)


Create sweep with ID: up19d36t
Sweep URL: https://wandb.ai/anirudhvr02-indian-institute-of-technology-madras/DA6401_Assignment1/sweeps/up19d36t


wandb: Agent Starting Run: hwwfo6q4 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.9999
wandb: 	criterion: cross_entropy
wandb: 	epochs: 5
wandb: 	epsilon: 1e-08
wandb: 	hidden_layers: 5
wandb: 	hidden_size: 32
wandb: 	input_size: 784
wandb: 	learning_rate: 0.0001
wandb: 	neurons: 128
wandb: 	num_layers: 1
wandb: 	optimizer: sgd
wandb: 	output_activation: softmax
wandb: 	output_size: 10
wandb: 	weight_decay: 5e-06
wandb: 	weight_initialization: random
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Wandb config: {'activation': 'sigmoid', 'batch_size': 16, 'beta': 0.9, 'beta1': 0.9, 'beta2': 0.9999, 'criterion': 'cross_entropy', 'epochs': 5, 'epsilon': 1e-08, 'hidden_layers': 5, 'hidden_size': 32, 'input_size': 784, 'learning_rate': 0.0001, 'neurons': 128, 'num_layers': 1, 'optimizer': 'sgd', 'output_activation': 'softmax', 'output_size': 10, 'weight_decay': 5e-06, 'weight_initialization': 'random'}


epoch,▁▃▅▆█
test_accuracy,▁
test_loss,▁
train_accuracy,▁▄▆▇█
train_loss,█▅▃▂▁
val_accuracy,▁▄▆▇█
val_loss,█▅▃▂▁
epoch,5
test_accuracy,0.1092
test_loss,3.36457
train_accuracy,0.11146


wandb: Agent Starting Run: n7oom298 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.9999
wandb: 	criterion: cross_entropy
wandb: 	epochs: 5
wandb: 	epsilon: 1e-08
wandb: 	hidden_layers: 4
wandb: 	hidden_size: 64
wandb: 	input_size: 784
wandb: 	learning_rate: 0.0001
wandb: 	neurons: 64
wandb: 	num_layers: 1
wandb: 	optimizer: rmsprop
wandb: 	output_activation: softmax
wandb: 	output_size: 10
wandb: 	weight_decay: 0
wandb: 	weight_initialization: random
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
test_loss,▁
train_accuracy,▁▂▃▄▅▆▆▇██
train_loss,█▇▅▄▄▃▂▂▁▁
val_accuracy,▁▂▃▄▅▅▆▇██
val_loss,█▇▅▄▄▃▂▂▁▁
epoch,10
test_accuracy,0.1587
test_loss,22.839
train_accuracy,0.15856


wandb: Agent Starting Run: oksoni3n with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	beta: 0.9
wandb: 	beta1: 0.9
wandb: 	beta2: 0.9999
wandb: 	criterion: cross_entropy
wandb: 	epochs: 5
wandb: 	epsilon: 1e-08
wandb: 	hidden_layers: 5
wandb: 	hidden_size: 32
wandb: 	input_size: 784
wandb: 	learning_rate: 0.001
wandb: 	neurons: 128
wandb: 	num_layers: 2
wandb: 	optimizer: adam
wandb: 	output_activation: softmax
wandb: 	output_size: 10
wandb: 	weight_decay: 5e-06
wandb: 	weight_initialization: random
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Wandb config: {'activation': 'relu', 'batch_size': 64, 'beta': 0.9, 'beta1': 0.9, 'beta2': 0.9999, 'criterion': 'cross_entropy', 'epochs': 5, 'epsilon': 1e-08, 'hidden_layers': 5, 'hidden_size': 32, 'input_size': 784, 'learning_rate': 0.001, 'neurons': 128, 'num_layers': 2, 'optimizer': 'adam', 'output_activation': 'softmax', 'output_size': 10, 'weight_decay': 5e-06, 'weight_initialization': 'random'}


epoch,▁▃▅▆█
test_accuracy,▁
test_loss,▁
train_accuracy,▁▇███
train_loss,█▂▁▁▁
val_accuracy,▁▇███
val_loss,█▂▁▁▁
epoch,5
test_accuracy,0.3761
test_loss,17.18231
train_accuracy,0.37698


wandb: Agent Starting Run: tuwvzhzk with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	beta: 0.7
wandb: 	beta1: 0.9
wandb: 	beta2: 0.9999
wandb: 	criterion: cross_entropy
wandb: 	epochs: 5
wandb: 	epsilon: 1e-08
wandb: 	hidden_layers: 3
wandb: 	hidden_size: 64
wandb: 	input_size: 784
wandb: 	learning_rate: 0.001
wandb: 	neurons: 128
wandb: 	num_layers: 3
wandb: 	optimizer: rmsprop
wandb: 	output_activation: softmax
wandb: 	output_size: 10
wandb: 	weight_decay: 0
wandb: 	weight_initialization: random
